In [108]:
import pandas as pd
from collections import defaultdict, Counter
import requests
import json 
import time
import numpy as np

In [133]:
data_folder = '/Users/wendy/Google Drive/Research/Tao-Lab/Data'
meddra_folder = data_folder + '/MedDRA_23_1_English/MedAscii'
umls_folder = data_folder + '/umls/2020AB-full/2020AB/META'
ohdsi_folder = data_folder + '/ohdsi-2021'

## MedDRA

In [319]:
def read_meddra_file(name):
    
    col_idx = [0,1]
    col_name =  [ f"{name.upper()}_CODE", f"{name.upper()}_NAME"]
    
    if name == 'pt':
        col_idx.extend([3])
        col_name.extend(['SOC_CODE'])
    if name == 'llt':
        col_idx.extend([2])
        col_name.extend(['PT_CODE'])
        
    df = pd.read_csv(meddra_folder + '/%s.asc' % name, 
                     usecols = col_idx,
                     names=col_name, 
                     sep='$')
    df[f"{name.upper()}_NAME"] = df[f"{name.upper()}_NAME"].str.lower()
    return df

In [115]:
df_pt = read_meddra_file('pt')
df_llt = read_meddra_file('llt')
df_soc = read_meddra_file('soc')

print(df_pt.shape, df_llt.shape, df_soc.shape)

(24571, 3) (82583, 3) (27, 2)


In [116]:
df_ptsoc = df_pt.merge(df_soc, on="SOC_CODE").drop(columns='SOC_CODE')
df_pt = df_pt.drop(columns='SOC_CODE')
df_ptsoc.head()

,PT_CODE,PT_NAME,SOC_NAME
0,10000002,11-beta-hydroxylase deficiency,"congenital, familial and genetic disorders"
1,10000013,"17,20-desmolase deficiency","congenital, familial and genetic disorders"
2,10000014,17-alpha-hydroxylase deficiency,"congenital, familial and genetic disorders"
3,10000020,"20,22-desmolase deficiency","congenital, familial and genetic disorders"
4,10000021,21-hydroxylase deficiency,"congenital, familial and genetic disorders"


In [117]:
df_ptllt = df_llt.merge(df_pt, how='left', on="PT_CODE").drop(columns='PT_CODE')
df_llt = df_pt.drop(columns='PT_CODE')
df_ptllt.head()

,LLT_CODE,LLT_NAME,PT_NAME
0,10000001,ventilation pneumonitis,hypersensitivity pneumonitis
1,10000002,11-beta-hydroxylase deficiency,11-beta-hydroxylase deficiency
2,10000003,11-oxysteroid activity incr,oxycorticosteroids increased
3,10000004,11-oxysteroid activity increased,oxycorticosteroids increased
4,10000005,17 ketosteroids urine,17 ketosteroids urine


## OHDSI

### Read files

In [255]:
%%time
def read_concepts():
#     with open(ohdsi_folder + "/CONCEPT.csv") as file:
#         concepts = np.asarray([line.strip().split('\t') for line in file])
#     # fill invalid_reason column if empty
#     for row in concepts:
#         if len(row) == 9 and len(row[8])==8:
#             row.append("NA")

#     concepts = np.asarray(concepts)
    concepts = pd.read_csv(ohdsi_folder + "/CONCEPT.csv", sep='\t')
    return concepts 

concepts = read_concepts()
concepts.head()

CPU times: user 2.17 s, sys: 90.8 ms, total: 2.26 s
Wall time: 2.28 s


/Users/wendy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (5,6,9) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,1146945,concept.concept_id,Metadata,CDM,Field,S,CDM1,20141111,20991231,NaN
1,1146954,concept.invalid_reason,Metadata,CDM,Field,S,CDM10,20141111,20991231,NaN
2,1147044,observation_period.observation_period_id,Metadata,CDM,Field,S,CDM100,20141111,20991231,NaN
3,1147045,observation_period.person_id,Metadata,CDM,Field,S,CDM101,20141111,20991231,NaN
4,1147046,observation_period.observation_period_start_date,Metadata,CDM,Field,S,CDM102,20141111,20991231,NaN


In [264]:
%%time
def read_concept_relationships():
    df = pd.read_csv(ohdsi_folder + "/CONCEPT_RELATIONSHIP.csv", sep='\t')
    df = df[["concept_id_1","concept_id_2","relationship_id"]]
    df.columns = ['id1','id2',"relation"]
    return df

relations = read_concept_relationships()
relations.head()

CPU times: user 4.55 s, sys: 231 ms, total: 4.78 s
Wall time: 4.81 s


,id1,id2,relation
0,44831619,44826991,Is a
1,44831619,44826995,Is a
2,44831620,380844,Maps to
3,44831620,437385,Maps to
4,44831620,439143,Maps to


### Extract vocabularies

In [285]:
print("ICD9CM - %s" % len(concepts.loc[concepts.vocabulary_id=='ICD9CM'].concept_name.unique()))
print("ICD10CM - %s" % len(concepts.loc[concepts.vocabulary_id=='ICD10CM'].concept_name.unique()))
print("MedDRA - %s" % len(concepts.loc[concepts.vocabulary_id=='MedDRA'].concept_name.unique()))

ICD9CM - 17345
ICD10CM - 96024
MedDRA - 84752


In [267]:
def retrieve_ohdsi_vocab(concepts, vocab_list, id_name):
    '''retrieve data downlaoded from OHDSI vocabularies (CONCEPT.CSV file)'''
#     ret = np.asarray([row for row in concepts if row[3] == vocab])
#     df = pd.DataFrame([[entry[0],entry[1],entry[3]] for entry in ret], 
#         columns = [id_name, "name",'vocab'])
    df = concepts.loc[concepts.vocabulary_id.isin(vocab_list)][['concept_id','concept_name','vocabulary_id']]
    df.columns = [id_name, "name",'vocab']
    return df


def retrieve_meddra(concepts, id_name, level):
#     ret = np.asarray([row for row in concepts if row[3] == "MedDRA" and row[4] == level])
#     df = pd.DataFrame([[entry[0],entry[1],entry[3]] for entry in ret], 
#         columns = [id_name, "name",'vocab'])
    df = concepts.loc[(concepts.vocabulary_id=="MedDRA") & (concepts.concept_class_id==level)]
    df = df[['concept_id','concept_name','vocabulary_id']]
    df.columns = [id_name, "name",'vocab']
    return df

In [286]:
%%time
icd9 = retrieve_ohdsi_vocab(concepts, ["ICD9CM"], "id1")
print(f"ICD9 {len(icd9.name.unique())}")

icd10 = retrieve_ohdsi_vocab(concepts, ["ICD10CM"], "id1")
print(f"ICD10 {len(icd10.name.unique())}")

mdr_llt = retrieve_meddra(concepts,"id1","LLT")
print(f"MedDRA LLT {len(mdr_llt.name.unique())}")

mdr_pt = retrieve_meddra(concepts,"id1","PT")
print(f"MedDRA PT {len(mdr_pt.name.unique())}")

snomed = retrieve_ohdsi_vocab(concepts,["SNOMED"],"id2")
print(f"SNOMED {len(snomed.name.unique())}")

ICD9 17345
ICD10 96024
MedDRA LLT 57981
MedDRA PT 24589
SNOMED 821919
CPU times: user 1.07 s, sys: 118 ms, total: 1.19 s
Wall time: 1.19 s


### Mapping

In [287]:
# get all maps-to relationships
def get_concept_relationships(file, relation):
    with open(file) as f:
        next(f)
        f = np.asarray([line.strip().split("\t") for line in f])
        df = pd.DataFrame([entry[:3] for entry in f if (entry[2] == relation)], 
            columns = ['id1','id2',"relation"])
        #df = pd.DataFrame([entry[:3] for entry in f], columns = ['id1','id2',"relation"])
    return df

def mapping_with_snomed(relations, vocab, name, snomed, relation):
    df_relation = relations.loc[relations.relation==relation]
    id_match = pd.merge(vocab, df_relation, how ='inner', on=['id1'])
    df = pd.merge(id_match, snomed, how ='inner', on=["id2"])
    
    df = df[["id1","name_x","id2"]]
    df = df.rename(index=str, columns={"id1": name+"_id", "name_x": name, "id2": "snomed_id"})
    return df

In [288]:
%%time
mdr_pt_snomed = mapping_with_snomed(relations, mdr_pt, "mdr", snomed, "MedDRA - SNOMED eq")
mdr_llt_snomed = mapping_with_snomed(relations, mdr_llt, "mdr", snomed, "MedDRA - SNOMED eq")
icd9_snomed = mapping_with_snomed(relations, icd9, "icd", snomed, "Maps to")
icd10_snomed = mapping_with_snomed(relations, icd10, "icd", snomed, "Maps to")

mdr_icd9_pt = pd.merge(mdr_pt_snomed, icd9_snomed, how="inner", on=["snomed_id"])
mdr_icd9_llt = pd.merge(mdr_llt_snomed, icd9_snomed, how="inner", on=["snomed_id"])
mdr_icd10_pt = pd.merge(mdr_pt_snomed, icd10_snomed, how="inner", on=["snomed_id"])
mdr_icd10_llt = pd.merge(mdr_llt_snomed, icd10_snomed, how="inner", on=["snomed_id"])

merged = pd.concat([mdr_icd9_pt[["mdr","icd"]],mdr_icd9_llt[["mdr","icd"]],
                        mdr_icd10_pt[["mdr","icd"]],mdr_icd10_llt[["mdr","icd"]]])
merged = merged.drop_duplicates(keep = "first")
merged.columns = ["MDR","ICD"]
merged.head()

CPU times: user 4.06 s, sys: 307 ms, total: 4.37 s
Wall time: 4.38 s


,MDR,ICD
0,Abdominal pain,Abdominal pain
1,Abdominal pain,"Abdominal pain, unspecified site"
2,Abdominal pain,"Abdominal pain, other specified site"
3,Abdominal rigidity,"Abdominal rigidity, periumbilic"
4,Abdominal rigidity,"Abdominal rigidity, generalized"


In [289]:
print("OHDSI MDR ICD9 PT: ", len(set(mdr_icd9_pt["mdr"].drop_duplicates().tolist())))
print("OHDSI MDR ICD10 PT: ", len(set(mdr_icd10_pt["mdr"].drop_duplicates().tolist())))
print("OHDSI MDR ICD9 LLT: ", len(set(mdr_icd9_llt["mdr"].drop_duplicates().tolist())))
print("OHDSI MDR ICD10 LLT: ", len(set(mdr_icd10_llt["mdr"].drop_duplicates().tolist())))
ohdsi_mdr_pt = pd.concat([mdr_icd9_pt,mdr_icd10_pt]).drop_duplicates()
ohdsi_mdr_llt = pd.concat([mdr_icd9_llt,mdr_icd10_llt]).drop_duplicates()
print("OHDSI MDR PT: ", len(set(ohdsi_mdr_pt["mdr"].drop_duplicates().tolist())))
print("OHDSI MDR LLT: ", len(set(ohdsi_mdr_llt["mdr"].drop_duplicates().tolist())))

OHDSI MDR ICD9 PT:  3308
OHDSI MDR ICD10 PT:  3542
OHDSI MDR ICD9 LLT:  200
OHDSI MDR ICD10 LLT:  213
OHDSI MDR PT:  4078
OHDSI MDR LLT:  246


In [290]:
merged["MDR"] = merged["MDR"].str.lower()
merged["ICD"] = merged["ICD"].str.lower()
ohdsi_mapping = merged
print(ohdsi_mapping.shape)
ohdsi_mapping.head()

(45163, 2)


,MDR,ICD
0,abdominal pain,abdominal pain
1,abdominal pain,"abdominal pain, unspecified site"
2,abdominal pain,"abdominal pain, other specified site"
3,abdominal rigidity,"abdominal rigidity, periumbilic"
4,abdominal rigidity,"abdominal rigidity, generalized"


## UMLS

2009-2016: /MRCONSO.RRF.aa <br/>
2017-2019: /MRCONSO.RRF <br/>

### Helper functions

In [275]:
%%time

# read MRCONSO.RRF file, get terms in English
def read_mrconso():
    term_df = pd.read_csv(umls_folder+"/MRCONSO.RRF", sep="|", usecols=[0,1,11,12,14])
    term_df.columns = ["CUI","LAT","SAB","TTY","STR"]
    term_df = term_df.loc[term_df["LAT"] == "ENG"]
    term_df['STR'] = term_df['STR'].str.lower()
    print(term_df.shape)
    return term_df

df = read_mrconso()
df.head()

(772101, 5)
CPU times: user 3.32 s, sys: 154 ms, total: 3.47 s
Wall time: 3.49 s


,CUI,LAT,SAB,TTY,STR
7,C0000727,ENG,ICD10,PT,acute abdomen
8,C0000727,ENG,ICD10AM,PT,acute abdomen
9,C0000727,ENG,ICD10CM,PT,acute abdomen
10,C0000727,ENG,ICD10CM,AB,acute abdomen
11,C0000727,ENG,MDR,LLT,acute abdomen


In [279]:
icd9 = df.loc[df['SAB']=="ICD9CM"].drop_duplicates()
icd10 = df.loc[df['SAB']=="ICD10CM"].drop_duplicates()
print(icd9.shape, icd10.shape)
mdr_pt =  df.loc[(df["SAB"] == 'MDR') & (df["TTY"] == "PT")].drop_duplicates()
mdr_llt =  df.loc[(df["SAB"] == 'MDR') & (df["TTY"] == "LLT")].drop_duplicates()
print("MDR PT", len(set(mdr_pt["STR"].tolist())))
print("MDR LLT", len(set(mdr_llt["STR"].tolist())))
mdr_pt.head()

(40809, 5) (202473, 5)
MDR PT 24313
MDR LLT 72587


,CUI,LAT,SAB,TTY,STR
12,C0000727,ENG,MDR,PT,acute abdomen
157,C0000731,ENG,MDR,PT,abdominal distension
238,C0000734,ENG,MDR,PT,abdominal mass
296,C0000735,ENG,MDR,PT,abdominal neoplasm
373,C0000737,ENG,MDR,PT,abdominal pain


In [280]:
def cui_mapping(mdr, icd):
    matches = pd.merge(mdr, icd, how='inner', on=["CUI"], suffixes=('_mdr', '_icd')).drop_duplicates()
    matches = matches.rename(columns={'STR_mdr': 'MDR', 'STR_icd': 'ICD'})
    return matches[["CUI","MDR","ICD"]]

# mapping percentage is calculated by how many MedDRA terms are in UMLS of that year
def get_count_info(matches, mdr):
    print(matches.shape)
    len_match_cui = len(set(matches["CUI"].tolist()))
    len_match_icd = len(set(matches["ICD"].tolist()))
    len_match_mdr = len(set(matches["MDR"].tolist()))
    len_mdr = len(set(mdr['STR'].tolist()))
    
    print("Mapped CUI: ", len_match_cui)
    print("Mapped ICD: ", len_match_icd)
    print("Mapped MDR: ", len_match_mdr)
    print("Terms in MDR: ", len_mdr)
    print("MDR Matched Percentage: ", float(len_match_mdr)/len_mdr)

In [281]:
map_icd9 = cui_mapping(mdr_pt, icd9)
map_icd10 = cui_mapping(mdr_pt, icd10)
combined_pt = pd.concat([map_icd9, map_icd10]).drop_duplicates()
print("---  ICD9 Mapping---")
get_count_info(map_icd9, mdr_pt)
print("\n---ICD10 Mapping---")
get_count_info(map_icd10, mdr_pt)
print("\n---Combined Mapping---")
get_count_info(combined_pt, mdr_pt)
combined_pt.head()

---  ICD9 Mapping---
(5748, 3)
Mapped CUI:  2671
Mapped ICD:  4465
Mapped MDR:  2788
Terms in MDR:  24313
MDR Matched Percentage:  0.11467116357504216

---ICD10 Mapping---
(8878, 3)
Mapped CUI:  3669
Mapped ICD:  5399
Mapped MDR:  3819
Terms in MDR:  24313
MDR Matched Percentage:  0.15707646115246987

---Combined Mapping---
(8879, 3)
Mapped CUI:  4438
Mapped ICD:  8462
Mapped MDR:  4609
Terms in MDR:  24313
MDR Matched Percentage:  0.18956936618270062


,CUI,MDR,ICD
0,C0000737,abdominal pain,abdominal pain
1,C0000737,abdominal pain,"abdominal pain, unspecified site"
2,C0000737,abdominal pain,abdmnal pain unspcf site
3,C0000768,foetal malformation,congenital anomalies
4,C0000768,foetal malformation,congenital anomaly nos


In [284]:
map_icd9 = cui_mapping(mdr_llt, icd9)
map_icd10 = cui_mapping(mdr_llt, icd10)
combined_llt = pd.concat([map_icd9, map_icd10]).drop_duplicates()
print("---  ICD9 Mapping---")
get_count_info(map_icd9, mdr_llt)
print("\n---ICD10 Mapping---")
get_count_info(map_icd10, mdr_llt)
print("\n---Combined Mapping---")
get_count_info(combined_llt, mdr_llt)
combined_llt.head()

---  ICD9 Mapping---
(27983, 3)
Mapped CUI:  6805
Mapped ICD:  12083
Mapped MDR:  13458
Terms in MDR:  72587
MDR Matched Percentage:  0.1854051000867924

---ICD10 Mapping---
(34292, 3)
Mapped CUI:  6196
Mapped ICD:  8761
Mapped MDR:  13851
Terms in MDR:  72587
MDR Matched Percentage:  0.190819292710816

---Combined Mapping---
(40496, 3)
Mapped CUI:  9457
Mapped ICD:  18390
Mapped MDR:  18664
Terms in MDR:  72587
MDR Matched Percentage:  0.2571259316406519


,CUI,MDR,ICD
0,C0000737,abdominal pain,abdominal pain
1,C0000737,abdominal pain,"abdominal pain, unspecified site"
2,C0000737,abdominal pain,abdmnal pain unspcf site
3,C0000737,pain abdominal,abdominal pain
4,C0000737,pain abdominal,"abdominal pain, unspecified site"


In [283]:
umls_mapping = pd.concat([combined_pt, combined_llt]).drop_duplicates()
print(umls_mapping.shape)

(40496, 3)


### UMLS Mapping

In [181]:
# def get_umls_mapping(mdr_pt_df):
#     df = read_mrconso()
    
#     # get ICD data
#     icd9 = df.loc[df['SAB'].str.contains("ICD9")].drop_duplicates()
#     icd10 = df.loc[df['SAB'].str.contains("ICD10")].drop_duplicates()
    
#     # get MedDRA data
#     mdr_pt =  df.loc[(df["SAB"] == 'MDR') & (df["TTY"] == "PT")].drop_duplicates()
#     mdr_llt =  df.loc[(df["SAB"] == 'MDR') & (df["TTY"] == "LLT")].drop_duplicates()
#     mdr_pt_num = len(set(mdr_pt["STR"].drop_duplicates().tolist()))
    
#     # MDR and ICD mapping
#     map_icd9_pt = cui_mapping(mdr_pt, icd9cm)
#     map_icd10_pt = cui_mapping(mdr_pt, icd10cm)
#     map_icd9_llt = cui_mapping(mdr_llt, icd9cm)
#     map_icd10_llt = cui_mapping(mdr_llt, icd10cm)
    
#     combined = pd.concat([map_icd9_pt, map_icd10_pt, map_icd9_llt, map_icd10_llt]).drop_duplicates()
    
# #     print("---  ICD9CM Mapping---")
# #     get_count_info(map_icd9, mdr_pt)
# #     print("\n---ICD10CM Mapping---")
# #     get_count_info(map_icd10, mdr_pt)
# #     print("\n---Combined Mapping---")
# #     get_count_info(combined, mdr_pt_df)
    
#     return combined, mdr_pt_num

In [182]:
# umls_mapping = get_umls_mapping()

## Combined Mapping

In [308]:
umls_mapping = umls_mapping[["MDR","ICD"]]
print(umls_mapping.shape)
umls_mapping.head()

(40496, 2)


,MDR,ICD
0,abdominal pain,abdominal pain
1,abdominal pain,"abdominal pain, unspecified site"
2,abdominal pain,abdmnal pain unspcf site
3,foetal malformation,congenital anomalies
4,foetal malformation,congenital anomaly nos


In [309]:
print(ohdsi_mapping.shape)
ohdsi_mapping.head()

(45163, 2)


,MDR,ICD
0,abdominal pain,abdominal pain
1,abdominal pain,"abdominal pain, unspecified site"
2,abdominal pain,"abdominal pain, other specified site"
3,abdominal rigidity,"abdominal rigidity, periumbilic"
4,abdominal rigidity,"abdominal rigidity, generalized"


In [310]:
print(df_ptllt.shape)
df_ptllt.head()

(82583, 3)


,LLT_CODE,LLT_NAME,PT_NAME
0,10000001,ventilation pneumonitis,hypersensitivity pneumonitis
1,10000002,11-beta-hydroxylase deficiency,11-beta-hydroxylase deficiency
2,10000003,11-oxysteroid activity incr,oxycorticosteroids increased
3,10000004,11-oxysteroid activity increased,oxycorticosteroids increased
4,10000005,17 ketosteroids urine,17 ketosteroids urine


In [311]:
print(df_ptsoc.shape)
df_ptsoc.head()

(24571, 3)


,PT_CODE,PT_NAME,SOC_NAME
0,10000002,11-beta-hydroxylase deficiency,"congenital, familial and genetic disorders"
1,10000013,"17,20-desmolase deficiency","congenital, familial and genetic disorders"
2,10000014,17-alpha-hydroxylase deficiency,"congenital, familial and genetic disorders"
3,10000020,"20,22-desmolase deficiency","congenital, familial and genetic disorders"
4,10000021,21-hydroxylase deficiency,"congenital, familial and genetic disorders"


In [312]:
umls_ohdsi_mapping = pd.concat([umls_mapping, ohdsi_mapping]).drop_duplicates()
print(umls_ohdsi_mapping.shape)
print("UMLS + OHDSI combined: ", len(set(umls_ohdsi_mapping["MDR"].drop_duplicates().tolist())))
umls_ohdsi_mapping.head()

(81654, 2)
UMLS + OHDSI combined:  19860


,MDR,ICD
0,abdominal pain,abdominal pain
1,abdominal pain,"abdominal pain, unspecified site"
2,abdominal pain,abdmnal pain unspcf site
3,foetal malformation,congenital anomalies
4,foetal malformation,congenital anomaly nos


In [313]:
pt_filtered = pd.merge(umls_ohdsi_mapping, df_ptllt, left_on="MDR", right_on="PT_NAME", how = "inner")[["MDR","ICD"]].drop_duplicates()
llt_filtered = pd.merge(umls_ohdsi_mapping,df_ptllt, left_on="MDR", right_on="LLT_NAME",how = "inner")[["MDR","ICD"]].drop_duplicates()
print(pt_filtered.shape, llt_filtered.shape)
print("Total PT filtered: ", len(set(pt_filtered["MDR"].drop_duplicates().tolist())))
print("Total LLT filtered: ", len(set(llt_filtered["MDR"].drop_duplicates().tolist())))
llt_filtered.head()

(47169, 2) (81654, 2)
Total PT filtered:  5726
Total LLT filtered:  19860


,MDR,ICD
0,abdominal pain,abdominal pain
1,abdominal pain,"abdominal pain, unspecified site"
2,abdominal pain,abdmnal pain unspcf site
3,abdominal pain,unspecified abdominal pain
4,abdominal pain,"abdominal pain, other specified site"


In [314]:
llt_to_pt_filtered = pd.merge(llt_filtered, df_ptllt, left_on="MDR", right_on="LLT_NAME")[["PT_NAME","ICD"]]
llt_to_pt_filtered.columns = ["MDR","ICD"]
result = pd.concat([pt_filtered,llt_to_pt_filtered]).drop_duplicates()
print(result.shape)
result.head()

(59029, 2)


,MDR,ICD
0,abdominal pain,abdominal pain
32,abdominal pain,"abdominal pain, unspecified site"
64,abdominal pain,abdmnal pain unspcf site
96,abdominal pain,unspecified abdominal pain
128,abdominal pain,"abdominal pain, other specified site"


In [315]:
len_match_icd = len(set(result["ICD"].tolist()))
len_match_mdr = len(set(result["MDR"].tolist()))
len_mdr = len(set(df_pt["PT_NAME"].drop_duplicates().tolist()))

print("Mapped ICD: ", len_match_icd)
print("Mapped MDR: ", len_match_mdr)
print("Terms in MDR: ", len_mdr)
print("MDR Matched Percentage: ", float(len_match_mdr)/len_mdr)

Mapped ICD:  49808
Mapped MDR:  6413
Terms in MDR:  24571
MDR Matched Percentage:  0.2609987383500875


## SOC Count

In [370]:
result_with_soc=result.merge(df_ptsoc, left_on="MDR", right_on="PT_NAME").drop(columns=['PT_NAME','PT_CODE'])
soc_mapped = result_with_soc[['MDR','SOC_NAME']].drop_duplicates().SOC_NAME.value_counts()
soc_mapped = soc_mapped.reset_index().rename(columns={"index": "SOC", "SOC_NAME": "Mapped"})[["SOC",'Mapped']]
soc_total = df_ptsoc[['PT_NAME','SOC_NAME']].SOC_NAME.value_counts().reset_index()
soc_total = soc_total.reset_index().rename(columns={"index": "SOC", "SOC_NAME": "Total"})[["SOC",'Total']]

In [371]:
soc_summary = soc_mapped.merge(soc_total, on = "SOC")
soc_summary["Not Mapped"] = soc_summary.Total - soc_summary.Mapped
soc_summary["Percentage"] = soc_summary.Mapped / soc_summary.Total
soc_summary

,SOC,Mapped,Total,Not Mapped,Percentage
0,infections and infestations,944,2056,1112,0.459144
1,"congenital, familial and genetic disorders",747,1647,900,0.453552
2,"neoplasms benign, malignant and unspecified (i...",503,2007,1504,0.250623
3,surgical and medical procedures,415,2347,1932,0.176821
4,"injury, poisoning and procedural complications",406,1294,888,0.313756
5,nervous system disorders,365,1021,656,0.357493
6,gastrointestinal disorders,313,919,606,0.340588
7,eye disorders,299,645,346,0.463566
8,investigations,267,5950,5683,0.044874
9,psychiatric disorders,233,551,318,0.422868


In [372]:
soc_summary.to_csv("/Users/wendy/Google Drive/Research/Tao-Lab/soc_counts.csv")

## UMLS over the years

In [139]:
for year in range(2009, 2020):
    umls_path = "/term_mapping/umls/" + str(year) + "AB"
    if year < 2017:
        mrconso_file = "/MRCONSO.RRF.aa"
    else:
        mrconso_file = "/MRCONSO.RRF"
    mrconso_path = umls_path + mrconso_file
    print("\n---" + str(year) + "---")
    combined_umls, mdr_pt_num = get_umls_mapping(mrconso_path, pt_df)
    
    # Filter using official MedDRA data
    pt_filtered = pd.merge(combined_umls, pt_llt_df, left_on="MDR", right_on="PT", how = "inner")[["MDR","ICD"]].drop_duplicates()
    llt_filtered = pd.merge(combined_umls,pt_llt_df, left_on="MDR", right_on="LLT",how = "inner")[["MDR","ICD"]].drop_duplicates()
#     print(pt_filtered.shape, llt_filtered.shape)
#     print("Total PT filtered: ", len(set(pt_filtered["MDR"].drop_duplicates().tolist())))
#     print("Total LLT filtered: ", len(set(llt_filtered["MDR"].drop_duplicates().tolist())))

    # converted LLT to PT
    llt_to_pt_filtered = pd.merge(llt_filtered, pt_llt_df, left_on="MDR", right_on="LLT")[["PT","ICD"]]
    llt_to_pt_filtered.columns = ["MDR","ICD"]
    result = pd.concat([pt_filtered,llt_to_pt_filtered]).drop_duplicates()
    
    len_match_icd = len(set(result["ICD"].drop_duplicates().tolist()))
    len_match_mdr = len(set(result["MDR"].drop_duplicates().tolist()))
#     len_mdr = len(set(combined_umls["MDR"].drop_duplicates().tolist()))

    print("Mapped ICD: ", len_match_icd)
    print("Mapped MDR: ", len_match_mdr)
    print("MDR PT in UMLS: ", mdr_pt_num)
    print("MDR Matched Percentage: ", float(len_match_mdr)/mdr_pt_num)


---2009---
(5635063, 5)
Mapped ICD:  4395
Mapped MDR:  2701
MDR PT in UMLS:  17976
MDR Matched Percentage:  0.15025589675122386

---2010---
(5546494, 5)
Mapped ICD:  7989
Mapped MDR:  4215
MDR PT in UMLS:  17833
MDR Matched Percentage:  0.23635955812258172

---2011---
(6063442, 5)
Mapped ICD:  8057
Mapped MDR:  4221
MDR PT in UMLS:  17873
MDR Matched Percentage:  0.2361662843395065

---2012---
(5996159, 5)
Mapped ICD:  8126
Mapped MDR:  4252
MDR PT in UMLS:  17962
MDR Matched Percentage:  0.23672196860037859

---2013---
(6087124, 5)
Mapped ICD:  8117
Mapped MDR:  4286
MDR PT in UMLS:  18102
MDR Matched Percentage:  0.2367694177438957

---2014---
(6063084, 5)
Mapped ICD:  8151
Mapped MDR:  4322
MDR PT in UMLS:  18183
MDR Matched Percentage:  0.23769454985425947

---2015---
(5315282, 5)
Mapped ICD:  8114
Mapped MDR:  4319
MDR PT in UMLS:  17899
MDR Matched Percentage:  0.24129839655846697

---2016---
(5305343, 5)
Mapped ICD:  17590
Mapped MDR:  5379
MDR PT in UMLS:  18000
MDR Matched Pe

In [306]:
# 2020AB

# Filter using official MedDRA data
pt_filtered = pd.merge(umls_mapping, df_ptllt, left_on="MDR", right_on="PT_NAME", how = "inner")[["MDR","ICD"]].drop_duplicates()
llt_filtered = pd.merge(umls_mapping,df_ptllt, left_on="MDR", right_on="LLT_NAME",how = "inner")[["MDR","ICD"]].drop_duplicates()
#     print(pt_filtered.shape, llt_filtered.shape)
#     print("Total PT filtered: ", len(set(pt_filtered["MDR"].drop_duplicates().tolist())))
#     print("Total LLT filtered: ", len(set(llt_filtered["MDR"].drop_duplicates().tolist())))

# converted LLT to PT
llt_to_pt_filtered = pd.merge(llt_filtered, df_ptllt, left_on="MDR", right_on="LLT_NAME")[["PT_NAME","ICD"]]
llt_to_pt_filtered.columns = ["MDR","ICD"]
result = pd.concat([pt_filtered,llt_to_pt_filtered]).drop_duplicates()

len_match_icd = len(set(result["ICD"].drop_duplicates().tolist()))
len_match_mdr = len(set(result["MDR"].drop_duplicates().tolist()))
#     len_mdr = len(set(combined_umls["MDR"].drop_duplicates().tolist()))

print("Mapped ICD: ", len_match_icd)
print("Mapped MDR: ", len_match_mdr)
print("MDR PT in UMLS: ", 24313)
print("MDR Matched Percentage: ", float(len_match_mdr)/24313)

Mapped ICD:  18390
Mapped MDR:  5711
MDR PT in UMLS:  24313
MDR Matched Percentage:  0.2348949121868959


## UMLS Mapping

In [4]:
umlsPath = "/Users/wendy/Google Drive/Research/term_mapping/2020AA/"
mrconso = pd.read_csv(umlsPath + "MRCONSO.RRF", sep="|")
mrrel = pd.read_csv(umlsPath + "MRREL.RRF", sep="|")
mrmap = pd.read_csv(umlsPath + "MRMAP.RRF", sep="|")
mrsmap = pd.read_csv(umlsPath + "MRSMAP.RRF", sep="|")

/Users/wendy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (9,10,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/wendy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (7,9,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/wendy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (5,6,8,14,16,20,21,22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/wendy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.


In [33]:
mrconso.CUI.value_counts()

(15479754, 19)

In [5]:
mrconso.columns = ["CUI",'LAT','TS','LUI','STT','SUI','ISPREF','AUI','SAUI','SCUI','SDUI',
                  'SAB','TTY','CODE','STR','SRL','SUPPRESS','CVF','NA']
mrrel.columns = ["CUI1","AUI1","STYPE1","REL","CUI2","AUI2","STYPE2","RELA",
                "RUI","SRUI","SAB","SL","RG","DIR","SUPPRESS","CVF","NA"]
mrmap.columns = ["MAPSETCUI","MAPSETSAB","MAPSUBSETID","MAPRANK","MAPID","MAPSID",
                "FROMID","FROMSID","FROMEXPR","FROMTYPE","FROMRULE","FROMRES",
                "REL","RELA","TOID","TOSID","TOEXPR","TOTYPE","TORULE","TORES",
                "MAPRULE","MAPRES","MAPTYPE","MAPATN","MAPATV","CVF","NA"]
mrsmap.columns = ["MAPSETCUI","MAPSETSAB","MAPID","MAPSID","FROMEXPR","FROMTYPE",
                 "REL","RELA","TOEXPR","TOTYPE","CVF","NA"]

In [6]:
icd9cm = mrconso.loc[mrconso["SAB"] == 'ICD9CM'].drop_duplicates()
icd10cm = mrconso.loc[mrconso["SAB"] == 'ICD10CM'].drop_duplicates()
mdr_pt =  mrconso.loc[(mrconso["SAB"] == 'MDR') & (mrconso["TTY"] == "PT")].drop_duplicates()
mdr_llt =  mrconso.loc[(mrconso["SAB"] == 'MDR') & (mrconso["TTY"] == "LLT")].drop_duplicates()

In [15]:
def get_value_percent(df, column):
    df_vc = df[column].value_counts().reset_index()
    df_vc["percent"]=df_vc[column]/len(df)
    return df_vc

### MRREL

In [7]:
mrrel_rel = mrrel.REL.value_counts().reset_index()
mrrel_rel["percent"]=mrrel.REL.value_counts().reset_index().REL/len(mrrel)
mrrel_rel

,index,REL,percent
0,SIB,37105040,0.422122
1,RO,22441644,0.255305
2,SY,6813962,0.077518
3,PAR,6772752,0.077050
4,CHD,6772752,0.077050
5,RQ,2759366,0.031392
6,RN,1977312,0.022495
7,RB,1977311,0.022495
8,QB,609243,0.006931
9,AQ,609243,0.006931


In [36]:
# MRREL mapped-to relationship details
get_value_percent(mrrel, "REL")

,index,REL,percent
0,SIB,37105040,0.422122
1,RO,22441644,0.255305
2,SY,6813962,0.077518
3,PAR,6772752,0.077050
4,CHD,6772752,0.077050
5,RQ,2759366,0.031392
6,RN,1977312,0.022495
7,RB,1977311,0.022495
8,QB,609243,0.006931
9,AQ,609243,0.006931


In [37]:
# MRREL RQ relationship
tst = mrrel.loc[mrrel.REL=="RQ"]
(tst.CUI1==tst.CUI2).reset_index()[0].value_counts().reset_index()

,index,0
0,False,1722976
1,True,1036390


In [39]:
# MRREL RQ rela3tionship - using CUI?
get_value_percent(tst[tst.CUI1!=tst.CUI2], 'STYPE1')

,index,STYPE1,percent
0,AUI,1435323,0.833049
1,CODE,237136,0.137632
2,SDUI,35921,0.020848
3,SCUI,11103,0.006444
4,CUI,3493,0.002027


In [42]:
get_value_percent(tst[tst.CUI183!=tst.CUI2], 'RELA')

,index,RELA,percent
0,classified_as,703424,0.408261
1,classifies,703424,0.408261
2,clinically_associated_with,29620,0.017191
3,has_allelic_variant,29201,0.016948
4,allelic_variant_of,29201,0.016948
5,mapped_to,21306,0.012366
6,mapped_from,21306,0.012366
7,isa,15517,0.009006
8,inverse_isa,15517,0.009006
9,primary_mapped_to,13769,0.007991


In [27]:
# MRREL SY relationship
tst = mrrel.loc[mrrel.REL=="SY"]
(tst.CUI1==tst.CUI2).reset_index()[0].value_counts().reset_index()

,index,0
0,True,6755032
1,False,58930


In [28]:
# MRREL SY relationship - using CUI?
get_value_percent(tst[tst.CUI1!=tst.CUI2], 'STYPE1')

,index,STYPE1,percent
0,SCUI,35454,0.601629
1,AUI,14986,0.254302
2,CODE,8490,0.144069


In [34]:
tst = mrrel.loc[mrrel.REL=="RL"]
(tst.CUI1==tst.CUI2).reset_index()[0].value_counts().reset_index()

,index,0
0,False,47658
1,True,15014


In [35]:
get_value_percent(tst[tst.CUI1!=tst.CUI2], 'STYPE1')

,index,STYPE1,percent
0,AUI,47658,1.0


### MRMAP

In [45]:
# MRMAP - using CUI?
print(get_value_percent(mrmap, 'FROMTYPE'))
print(get_value_percent(mrmap, 'TOTYPE'))
print(get_value_percent(mrmap.loc[mrmap.FROMTYPE=='CUI'], 'TOTYPE'))

  index  FROMTYPE   percent
0  SCUI    651670  0.939103
1  CODE     33944  0.048916
2   CUI      8314  0.011981
                     index  TOTYPE   percent
0                     SDUI  315483  0.454634
1                     CODE  269145  0.387857
2                     SCUI   53137  0.076574
3   BOOLEAN_EXPRESSION_STR    8314  0.011981
4  BOOLEAN_EXPRESSION_SDUI    1055  0.001520
5  BOOLEAN_EXPRESSION_CODE       1  0.000001
                    index  TOTYPE  percent
0  BOOLEAN_EXPRESSION_STR    8314      1.0


In [159]:
print(mrmap.RELA.value_counts())
print("---")
print(mrmap.loc[mrmap.REL=="RO"].RELA.value_counts())
print("---")
print(mrmap.loc[mrmap.REL=="RQ"].RELA.value_counts())
print("---")
print(mrmap.loc[mrmap.REL=="SY"].RELA.value_counts())

mapped_to        317979
classified_as    260659
same_as           49339
Name: RELA, dtype: int64
---
mapped_to    303122
Name: RELA, dtype: int64
---
classified_as    260659
mapped_to         12839
Name: RELA, dtype: int64
---
same_as    49339
Name: RELA, dtype: int64


In [43]:
get_value_percent(tst[tst.CUI1!=tst.CUI2], 'STYPE1')

,MAPSETCUI,MAPSETSAB,MAPSUBSETID,MAPRANK,MAPID,MAPSID,FROMID,FROMSID,FROMEXPR,FROMTYPE,...,TOTYPE,TORULE,TORES,MAPRULE,MAPRES,MAPTYPE,MAPATN,MAPATV,CVF,NA
0,C1306694,MTH,NaN,NaN,AT102971858,NaN,C0276253,NaN,C0276253,CUI,...,BOOLEAN_EXPRESSION_STR,NaN,NaN,NaN,NaN,ATX,NaN,NaN,NaN,NaN
1,C1306694,MTH,NaN,NaN,AT102971859,NaN,C0409780,NaN,C0409780,CUI,...,BOOLEAN_EXPRESSION_STR,NaN,NaN,NaN,NaN,ATX,NaN,NaN,NaN,NaN
2,C1306694,MTH,NaN,NaN,AT102971861,NaN,C1706094,NaN,C1706094,CUI,...,BOOLEAN_EXPRESSION_STR,NaN,NaN,NaN,NaN,ATX,NaN,NaN,NaN,NaN
3,C1306694,MTH,NaN,NaN,AT102971862,NaN,C1706094,NaN,C1706094,CUI,...,BOOLEAN_EXPRESSION_STR,NaN,NaN,NaN,NaN,ATX,NaN,NaN,NaN,NaN
4,C1306694,MTH,NaN,NaN,AT102971863,NaN,C0180739,NaN,C0180739,CUI,...,BOOLEAN_EXPRESSION_STR,NaN,NaN,NaN,NaN,ATX,NaN,NaN,NaN,NaN


In [47]:
icd9cm_from = pd.merge(mrmap,icd9cm,left_on = "FROMID",right_on = "CUI")
icd10cm_from = pd.merge(mrmap,icd10cm,left_on = "FROMID",right_on = "CUI")
mdr_pt_from = pd.merge(mrmap,mdr_pt,left_on = "FROMID",right_on = "CUI")
mdr_llt_from = pd.merge(mrmap,mdr_llt,left_on = "FROMID",right_on = "CUI")
print(icd9cm_from.shape, icd10cm_from.shape, mdr_pt_from.shape,mdr_llt_from.shape)

(6184, 46) (2224, 46) (979, 46) (3633, 46)


In [161]:
print(icd9cm_from.TOTYPE.value_counts())
print(icd10cm_from.TOTYPE.value_counts())
print(mdr_pt_from.TOTYPE.value_counts())
print(mdr_llt_from.TOTYPE.value_counts())
icd9cm_from[["FROMEXPR","TOEXPR","STR"]]

BOOLEAN_EXPRESSION_STR    6174
Name: TOTYPE, dtype: int64
BOOLEAN_EXPRESSION_STR    2188
Name: TOTYPE, dtype: int64
BOOLEAN_EXPRESSION_STR    979
Name: TOTYPE, dtype: int64
BOOLEAN_EXPRESSION_STR    3633
Name: TOTYPE, dtype: int64


,FROMEXPR,TOEXPR,STR
0,C0276253,<Pneumonia> AND <Cytomegalovirus Infections>,pneumonia in cytomegalic inclusion disease
1,C0276253,<Pneumonia> AND <Cytomegalovirus Infections>,pneum w cytomeg incl dis
2,C0409780,<Synovitis> AND <Hand>,"villonodular synovitis, hand"
3,C0409780,<Synovitis> AND <Hand>,villonod synovit-hand
4,C1533661,<Arthroscopy> AND <Wrist Joint>,"arthroscopy, wrist"
...,...,...,...
6169,C0026995,<Spinal cord>/<Radiography>,contrast myelogram
6170,C0015423,<Eyelids>/<Diseases>,unspecified disorder of eyelid
6171,C0015423,<Eyelids>/<Diseases>,disorder of eyelid nos
6172,C0031707,<Phosphorus>/<Metabolism>/<Disorders>,disorders of phosphorus metabolism


In [173]:
mrconso.CODE.value_counts()

NOCODE           247844
TCGA               2286
SDTM-LBTESTCD      1992
SDTM-LBTEST        1992
101794              582
                  ...  
2251620               1
2428390               1
0SSQ                  1
709972                1
LP16434-0             1
Name: CODE, Length: 4536727, dtype: int64